In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "who are you?"},
]


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B")
tokenizer = AutoTokenizer.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B")
model.config.pad_token_id = model.config.eos_token_id
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=50)
pipe(messages)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'Hello! I am an AI assistant designed to provide information, answer questions, and assist with tasks to the best of my abilities. I am a machine learning model trained on a vast amount of data, which enables me to understand and respond to a wide'}]}]

In [3]:
!pip install transformers accelerate


In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B")
model = AutoModelForCausalLM.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
input_text = "유해 요소: ['구제불능']. 올바른 문장:"
input_ids = tokenizer.encode(input_text, return_tensors="pt")


In [8]:
output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    early_stopping=True,
    temperature=0.3,
    top_k=50,
    top_p=0.9                # 누적 확률이 p 이하인 단어만 고려
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


유해 요소: ['구제불능']. 올바른 문장: '구제불능'은 유해 요소로 사용됩니다. 예: '구제불능'은 유해 요소로 사용됩니다.


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B")
model = AutoModelForCausalLM.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B")

tokenizer.pad_token = tokenizer.eos_token

input_text = """
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경해줘.
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장:"""


inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

output = model.generate(
    inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_new_tokens=50,
    temperature=0.7,
    top_k=50,
    top_p=0.9
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경해줘.
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그런 인간은 정말 도움이 되는 사람이다.

이제 '구제불능'은 '도움이 되는 사람'으로 변경되었다. 구제불능은 특정 상황에서 도움을 받지 못하거나 도움


In [10]:
# 초기 생성
input_text = """
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장:"""

initial_output = model.generate(
    tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)['input_ids'],
    max_new_tokens=50,
    temperature=0.7,
    top_k=50,
    top_p=0.9
)

initial_result = tokenizer.decode(initial_output[0], skip_special_tokens=True)
print("Initial Result:", initial_result)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Initial Result: 
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그 인간은 정말 도움이 안 되고 있는 사람이다.

이제 문장을 다른 문맥에 맞게 다시 사용해보겠습니다.
기존 문장: 그들은 그 집에 도움이 안 되는 사람들이다.
수정


In [11]:
# Self-reflection 단계
reflection_input = f"""
생성된 문장은 다음과 같습니다:
"{initial_result}"
이 문장은 원래 문장의 의미를 보존하며 긍정적으로 변경되었나?
문맥에 맞지 않다면, 올바른 표현으로 다시 수정해줘.
"""

reflection_output = model.generate(
    tokenizer(reflection_input, return_tensors="pt", padding=True, truncation=True)['input_ids'],
    max_new_tokens=50,
    temperature=0.7,
    top_k=50,
    top_p=0.9
)

final_result = tokenizer.decode(reflection_output[0], skip_special_tokens=True)
print("Final Result after Reflection:", final_result)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Final Result after Reflection: 
생성된 문장은 다음과 같습니다:
"
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그 인간은 정말 도움이 안 되고 있는 사람이다.

이제 문장을 다른 문맥에 맞게 다시 사용해보겠습니다.
기존 문장: 그들은 그 집에 도움이 안 되는 사람들이다.
수정"
이 문장은 원래 문장의 의미를 보존하며 긍정적으로 변경되었나? 
문맥에 맞지 않다면, 올바른 표현으로 다시 수정해줘.
수정된 문장: 그들은 그 집에 도움이 안 되고 있는 사람들이다.

이제 문장의 의미를 살펴보겠습니다.
기존 문장: 그런 인간은 정말 구제불능이다.



In [15]:
#loop 적용
for _ in range(3):
    reflection_input = f"""
    이전 결과: "{initial_result}"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나
    바꿀 필요가 없다면 새로운 생성을 멈춰줘.
    """
    reflection_output = model.generate(
        tokenizer(reflection_input, return_tensors="pt", padding=True, truncation=True)['input_ids'],
        max_new_tokens=50,
        temperature=0.7,
        top_k=50,
        top_p=0.9
    )
    initial_result = tokenizer.decode(reflection_output[0], skip_special_tokens=True)
    print("Updated Result:", initial_result)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Updated Result: 
    이전 결과: "
    이전 결과: "
    이전 결과: "
    이전 결과: "
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그 인간은 정말 도움이 안 되고 있는 사람이다.

이제 문장을 다른 문맥에 맞게 다시 사용해보겠습니다.
기존 문장: 그들은 그 집에 도움이 안 되는 사람들이다.
수정"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘.
     예: "그들은 그 집에 도움이 안 되는 사람들"이 맞습니다.

이제 문장을 다른 문맥에 맞게 다시 사용해보겠습니다.
기존 문장: 그들은 그 집에 도움이 안 되는"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘.
     예: "그들은 그 집에 도움이 안 되는 사람들"이 맞습니다.
이제 문장을 다른 문맥에 맞게 다시 사용해보겠습니다.
기존 문장: 그들은 그 집에 도움이 안 되는"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘.
     예: "그들은 그 집에 도움이 안 되는 사람들"이 맞습니다.
이제 문장을 다른 문맥에 맞게 다시 사용해보겠습니다.
기존 문장: 그들은 그 집에 도움이 안 된"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 
    바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     예: "그들은 그 집에 도움이 안 된"이 맞습니다.
이제 문장을 다른 문맥에 맞게 다시 사용해보겠습니다.
기존 문장: 그들은 그 집에 도움이 안 된"
   


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Updated Result: 
    이전 결과: "
    이전 결과: "
    이전 결과: "
    이전 결과: "
    이전 결과: "
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그 인간은 정말 도움이 안 되고 있는 사람이다.

이제 문장을 다른 문맥에 맞게 다시 사용해보겠습니다.
기존 문장: 그들은 그 집에 도움이 안 되는 사람들이다.
수정"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘.
     예: "그들은 그 집에 도움이 안 되는 사람들"이 맞습니다.

이제 문장을 다른 문맥에 맞게 다시 사용해보겠습니다.
기존 문장: 그들은 그 집에 도움이 안 되는"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘.
     예: "그들은 그 집에 도움이 안 되는 사람들"이 맞습니다.
이제 문장을 다른 문맥에 맞게 다시 사용해보겠습니다.
기존 문장: 그들은 그 집에 도움이 안 되는"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘.
     예: "그들은 그 집에 도움이 안 되는 사람들"이 맞습니다.
이제 문장을 다른 문맥에 맞게 다시 사용해보겠습니다.
기존 문장: 그들은 그 집에 도움이 안 된"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 
    바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     예: "그들은 그 집에 도움이 안 된"이 맞습니다.
이제 문장을 다른 문맥에 맞게 다시 사용해보겠습니다.
기존 문장: 그들은 그 집에 도움이 안 된"
   "
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 
    바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     예: "그들은 그 집에 도움이 안 된"이 맞습니다.
이제 문장을 다른 문맥에 맞게 다시 사용해보겠습니다.
기존 문

KeyboardInterrupt: 

In [ ]:
# 입력에서 가장 최근 context 유지해보기
reflection_input = f"""
이전 생성된 문장: "{initial_result}"
문맥이 맞지 않거나 의미가 왜곡된 경우, 아래 조건에 따라 수정하세요:
1. "구제불능"이라는 표현을 완화하고 문맥에 맞도록 수정하세요.
2. 기존 문장의 원래 의미를 유지하세요.

새로운 문장:"""


In [ ]:
output = model.generate(
    tokenizer(reflection_input, return_tensors="pt", padding=True, truncation=True)['input_ids'],
    max_new_tokens=50,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    no_repeat_ngram_size=3
)


제한걸기


In [ ]:
prev_result = None  # 이전 결과 초기화

for i in range(5):  # 최대 5번 반복
    reflection_input = f"""
    이전 결과: "{initial_result}"
    조건: "구제불능"을 완화하여 문맥에 맞는 문장으로 수정하세요.
    """
    output = model.generate(
        tokenizer(reflection_input, return_tensors="pt", padding=True, truncation=True)['input_ids'],
        max_new_tokens=50,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        no_repeat_ngram_size=3
    )
    new_result = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # 동일한 결과가 반복되면 종료
    if new_result == prev_result:
        print("No further improvement, stopping.")
        break

    # 결과 업데이트
    prev_result = new_result
    print(f"Iteration {i+1} Result:", new_result)
